In [1]:
!pip install AugmentedSocialScientist pandas numpy wordcloud matplotlib spacy nltk
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 38.3 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


REMARQUE : BESOIN D'AVOIR FAIT TOURNE DESCRIPTION ANNOTATIONS POUR CREER LE FICHIER AVEC LES PHRASES !

In [1]:
import numpy as np
import random
import torch
import sklearn
random_seed = 1968
# Set random seed for NumPy
np.random.seed(random_seed)

# Set random seed for Python built-in random module
random.seed(random_seed)

# Set random seed for PyTorch
torch.manual_seed(random_seed)

# Set random seed for Scikit-learn
sklearn.utils.check_random_state(random_seed)


RandomState(MT19937) at 0x7FB7A99EC240

In [3]:
import pandas as pd

current_annotations_file = "../../data/intermediate/annotations/personne_withSentence_1400annotations_2024-06-03_ActiveTigger.csv"

annotations_files = pd.read_csv(current_annotations_file, dtype = {'sentence_id':'int', 
'Personne':'str','sentence':'str'}).set_index('sentence_id')

recat = {
    "Corps humain":"Corps humain",
    "Genre/espèce humaine":"Genre/espèce humaine",
    "Individu ou sujet" : "Individu ou sujet",
    "Personne à protéger":"Personne droits et devoirs",
    "Personne responsable et autonome":"Personne droits et devoirs",
    "Personne relationnelle":"Personne relationnelle",
    "Sentiment humain":"Genre/espèce humaine",
    "Poubelle" : "Poubelle"
}

frecat = lambda x: recat[x]
annotations_files['recat']=annotations_files["Personne"].apply(frecat)


annotations = annotations_files[~(annotations_files["recat"]=="Poubelle")]

annotations['label'] = annotations['recat']

annotations['text'] = annotations['sentence']

annotations.sample(5)

/tmp/ipykernel_33036/3703100089.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotations['label'] = annotations['recat']
/tmp/ipykernel_33036/3703100089.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotations['text'] = annotations['sentence']


,Personne,sentence,recat,label,text
sentence_id,,,,,
1190093,Personne responsable et autonome,"En revanche, \nleur rapidité et leur relative ...",Personne droits et devoirs,Personne droits et devoirs,"En revanche, \nleur rapidité et leur relative ..."
870236,Personne relationnelle,Refus par un tiers 8 \n \nIl est de nombreuses...,Personne relationnelle,Personne relationnelle,Refus par un tiers 8 \n \nIl est de nombreuses...
1210148,Sentiment humain,"Elle est une réalité \nhumaine, qui tient à la...",Genre/espèce humaine,Genre/espèce humaine,"Elle est une réalité \nhumaine, qui tient à la..."
860089,Personne responsable et autonome,"Ainsi l’ensemble de ces inconvénients qui, pou...",Personne droits et devoirs,Personne droits et devoirs,"Ainsi l’ensemble de ces inconvénients qui, pou..."
1130224,Corps humain,21415 »30 Voir l’avis n°112 du CCNE « Une réf...,Corps humain,Corps humain,21415 »30 Voir l’avis n°112 du CCNE « Une réf...


# Bert classification multicat

In [4]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from torch import cuda

cuda.get_device_name(0)

'NVIDIA A2'

In [5]:
from sklearn.model_selection import train_test_split
 
train_data, test_data = train_test_split(annotations, train_size=0.80, random_state=random_seed, stratify=annotations["Personne"])

In [6]:
from AugmentedSocialScientist.models import Camembert

bert = Camembert()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


There are 1 GPU(s) available.
We will use GPU 0: NVIDIA A2


In [7]:
batch_size = 16
train_loader = bert.encode(train_data.text.values,
train_data.label.values, batch_size=batch_size)

  0%|          | 0/1110 [00:00<?, ?it/s]

  0%|          | 0/1110 [00:00<?, ?it/s]

label ids: {'Corps humain': 0, 'Genre/espèce humaine': 1, 'Individu ou sujet': 2, 'Personne droits et devoirs': 3, 'Personne relationnelle': 4}


In [8]:
test_loader = bert.encode(test_data.text.values,
test_data.label.values, batch_size=batch_size)

  0%|          | 0/278 [00:00<?, ?it/s]

  0%|          | 0/278 [00:00<?, ?it/s]

label ids: {'Corps humain': 0, 'Genre/espèce humaine': 1, 'Individu ou sujet': 2, 'Personne droits et devoirs': 3, 'Personne relationnelle': 4}


In [9]:
scores = bert.run_training(
    train_loader,
    test_loader,
    n_epochs=5,
    lr=2e-5,
    random_state=random_seed,
    save_model_as='multicat'
    )

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/opt/mamba/lib/python3.12/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



======== Epoch 1 / 5 ========
Training...
  Batch    40  of     70.    Elapsed: 0:00:49.

  Average training loss: 1.53
  Training took: 0:01:25

Running Validation...

  Average test loss: 1.34
  Validation took: 0:00:07
                            precision    recall  f1-score   support

              Corps humain       0.60      0.81      0.69        48
      Genre/espèce humaine       0.65      0.43      0.52        51
         Individu ou sujet       0.00      0.00      0.00        50
Personne droits et devoirs       0.44      0.94      0.60        84
    Personne relationnelle       0.00      0.00      0.00        45

                  accuracy                           0.50       278
                 macro avg       0.34      0.44      0.36       278
              weighted avg       0.36      0.50      0.40       278


======== Epoch 2 / 5 ========
Training...


/opt/mamba/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/mamba/lib/python3.12/site-packages/sklearn/m

  Batch    40  of     70.    Elapsed: 0:00:49.

  Average training loss: 1.22
  Training took: 0:01:25

Running Validation...

  Average test loss: 1.08
  Validation took: 0:00:07
                            precision    recall  f1-score   support

              Corps humain       0.80      0.81      0.80        48
      Genre/espèce humaine       0.73      0.78      0.75        51
         Individu ou sujet       0.71      0.10      0.18        50
Personne droits et devoirs       0.52      0.90      0.66        84
    Personne relationnelle       0.80      0.36      0.49        45

                  accuracy                           0.63       278
                 macro avg       0.71      0.59      0.58       278
              weighted avg       0.69      0.63      0.59       278


======== Epoch 3 / 5 ========
Training...
  Batch    40  of     70.    Elapsed: 0:00:49.

  Average training loss: 0.99
  Training took: 0:01:25

Running Validation...

  Average test loss: 0.97
  Validat

In [10]:
categories = ["Corps humain","Genre/espèce humaine","Individu ou sujet",
            "Personne relationnelle","Personne droits et devoirs"]
pd.DataFrame({
    "category" : categories,
    "precision" : scores[0],
    "recall" : scores[1],
    "f1-score" : scores[2],
    "support" : scores[3]
}).set_index("category").to_csv("./models/multicat/scores.csv")

In [11]:
pd.DataFrame({
    "category" : categories,
    "precision" : scores[0],
    "recall" : scores[1],
    "f1-score" : scores[2],
    "support" : scores[3]
}).set_index("category")

,precision,recall,f1-score,support
category,,,,
Corps humain,0.780000,0.812500,0.795918,48
Genre/espèce humaine,0.722222,0.764706,0.742857,51
Individu ou sujet,0.656250,0.420000,0.512195,50
Personne relationnelle,0.622449,0.726190,0.670330,84
Personne droits et devoirs,0.659091,0.644444,0.651685,45


In [12]:
import os
import zipfile

def zip_folder(folder_path, zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Create a relative path for the file inside the zip archive
                file_path = os.path.join(root, file)
                relative_path = os.path.relpath(file_path, folder_path)
                zipf.write(file_path, relative_path)
    print(f"Folder '{folder_path}' successfully zipped to '{zip_file_path}'")

# Example usage
folder_path = '/models/multicat'
zip_file_path = 'multicat.zip'
zip_folder(folder_path, zip_file_path)


Folder '/models/multicat' successfully zipped to 'multicat.zip'


# Prediction

In [ ]:
import numpy as np
pred_data = annotations_file[annotations_file["Personne"].isna()][["text"]].dropna()
pred_loader = bert.encode(pred_data.text.values, batch_size=batch_size)
pred_proba = bert.predict_with_model(pred_loader, model_path='./models/clickbait')
pred_data['pred_label'] = np.argmax(pred_proba, axis=1)
pred_data['pred_proba'] = np.max(pred_proba, axis=1)

KeyError: 'label'

In [ ]:
pred_data["pred_proba"].describe()

count    598.000000
mean       0.292393
std        0.072298
min        0.189324
25%        0.240241
50%        0.262016
75%        0.335396
max        0.440672
Name: pred_proba, dtype: float64

In [ ]:
pd.set_option('display.max_colwidth', None)  # Use None to show all the text, or set a large number

pred_data

,text,pred_label,pred_proba
sentence_id,,,
1120067,"Et à l’importance nouvelle que prenait le projet parental répondait l’émergence d’une forme \nnouvelle de responsabilité médicale et sociale quant au devenir de cet embryon avant son \ntransfert dans le corps de la mère. C’est dans ce contexte radicalement nouveau de début de la vie humaine qu’est apparue la \nquestion nouvelle du devenir de ces embryons in vitro lorsqu’ils n’étaient pas transférés dans \nle corps de leur mère. Il y a aujourd’hui au moins trois circonstances qui vont conduire à une décision \nmédicale d’absence de transfert d’un embryon créé in vitro par AMP dans le cadre d’un \nprojet parental :\n1. lorsqu’une anomalie majeure ou une interruption du développement d’un embryon est \nmanifeste in vitro, avant son transfert dans le corps de sa mère ;\n2. lorsqu’au cours d’un diagnostic préimplantatoire (DPI), un embryon se révèle être porteur \nde la séquence génétique en cause dans une maladie familiale d’une particulière gravité et \n4\nincurable au moment du diagnostic, séquence génétique dont la recherche a motivé la \nréalisation du DPI 1.",0,0.435750
900167,"9\ndemande d’un enfant devenu majeur le rapprochement des parties concernées et à \nrésoudre des dilemmes dans des situations difficiles. Avant d’éviter toute ambiguïté, le \nnouvel article L 1477 du Code de l’action sociale et des familles précise que l’accès d’une \npersonne à ses origines est sans effet sur l’état civil et la filiation, « il ne fait naître ni \ndroit ni obligation au profit ou à la charge de qui que ce soit ». La Cour europé enne des droits de l’hom me a rejeté le 13 février 2003 à Strasbourg, la requête \nd’une personne de 37 ans, abandonné e avec demande de secret et adoptée par la suite, qui \nalléguait que le secret de sa naissance et l’impossibilité qui en résultait pour elle de conna ître \nses origines constituaient une violation de ses droits garantis par l’article 8 de la Conve ntion \net une discrimination contraire à l’article 14 de la Conve ntion.",4,0.258446
1290186,"2.1. Mutations du concept de santé : progrès et nouvelles \nfragilités \n \n(i) Les maladies chroniques, un nouveau paradigme pour notre système de \nsanté \n \nLes maladies dites « chroniques », sont fréquentes - elles affectent en France \nenviron 15 millions de personnes, soit 20 % de la population, dont 9 millions sont \ndéclarées en affections de longue durée (ALD). Le nombre de maladies \nchroniques et de patients atteints est en augmentation constante, conséquence de \nl’allongement de la durée de vie, de l’efficacité des traitements (VIH, cancers , \nmaladies cardiovasculaires), mais aussi de multiples facteurs environnementaux \n« pathogènes », regroupés sous la notion nouvelle « d’exposome » qui souligne le \nlien entre santé et environnement.",2,0.251902
250110,"Le droit de savoir\nIl faut tenir chaque sujet au courant des éventuels résultats et l'informer clairement de leur\nsignification. Mais la connaissance peut limiter l'autonomie de l'individu. Ce peut être la situation dans les maladies monogéniques où la connaissance peut modifier\nle comportement d'un sujet, par exemple dans les maladies liées au sexe, connaissance\nchez une femme d'être conductrice de la maladie (myopathie, hémophilie...), dans les\nmaladies dominantes, connaissance d'être porteur du gène muté, donc d'être atteint dans\nl'avenir (chorée de Huntington).",0,0.322948
1330246,"Il apparait important, quelle que soit la relative facilitée de leur mise en œuvre, \nde développer des approches expérimentales visant à les rendre plus sûres, voire \nréversibles, et de les encadrer lors de leur application au monde du vivant. - Les applications à tout le vivant non humain des modifications ciblées du génome sont \nsource de bénéfices potentiels indéniables. Cependant il convient de considérer le bienêtre animal et d’éventuelles conséquences non maîtrisables, voire dramatiques, comme \nle bouleversement d’écosystèmes et d’ensembles